# Desafio ASA -2023
**Participante:** Ludmila Dias

**Tipo de Modelo Utilizado:** Regressão Logística

**Cenário:**

A ArcelorMittal está procurando formas de melhorar sua identificação de defeitos nas placas de aço. As placas são produzidas no lingotamento contínuo, após um processo de várias etapas que começa nos altos-fornos. Por causa da complexidade do processo, diversos defeitos podem ocorrer na placa produzida. Nosso especialista gerou um conjunto de dados contendo dois defeitos que ele gostaria de distinguir com maior exatidão. Todos os dados foram obtidos a partir de sensores automatizados ou imagens de câmeras, que identificam dimensões e características da placa e do defeito.


 **Objetivo:**

 Encontrar insights a partir dos dados e auxiliar o time de qualidade da ArcelorMittal a identificar se o defeito encontrado na placa é do tipo 0 ou do tipo 1.

## Configurações Iniciais


In [ ]:
%pylab inline

import pandas as pd
import plotly.express as px
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning:

pylab import has clobbered these variables: ['rec', 'var']
`%matplotlib` prevents importing * from pylab and numpy



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
random.seed(38) # Usando o 38 para não perder o costume ^^

## Leitura de Dados

Descrição sobre cada coluna de acordo com a documentação.
<br>*Com exceção de 'slab_thickness' que não foi relatado sobre*
- min_x_defect – Coordenada x inicial do defeito
- max_x_ defect – Coordenada x final do defeito
- min_y_ defect – Coordenada y inicial do defeito
- max_y_ defect – Coordenada y final do defeito
- area_pixels – Total de pixels presentes na placa
- slab_width – Largura da placa (eixo X)
- slab_length – Comprimento da placa (eixo Y)
- slab_thickness - Grossura ou Densidade da placa (?)
- sum_pixel_luminosity – Soma da luminosidade dos pixels
- min_pixel_luminosity – Mínima luminosidade dos pixels
- max_pixel_luminosity – Máxima luminosidade dos pixels
- conveyer_width – Largura da esteira (correia) transportadora (eixo X)
- type_of_steel – Identifica a classe do aço: pode pertencer à classe A300 ou A400
- defect_type – Tipo de defeito da classe. Pode ser do tipo 0 ou do tipo 1.


Importando arquivo csv do Google Drive.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASA 2023/desafioASA/estudo_de_caso.csv')
df.head(10)

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
0,38.0,49.0,735612.0,735624.0,113.0,11.0,12.0,12652.0,93.0,130.0,1707.0,100.0,TypeOfSteel_A300,1
1,1252.0,1348.0,355940.0,356016.0,1812.0,119.0,135.0,196003.0,NaN,132.0,1687.0,80.0,TypeOfSteel_A300,1
2,193.0,210.0,612201.0,612252.0,588.0,18.0,51.0,62182.0,73.0,135.0,1353.0,290.0,TypeOfSteel_A400,1
3,1159.0,1170.0,32914.0,32926.0,106.0,11.0,12.0,12792.0,100.0,134.0,1353.0,185.0,TypeOfSteel_A400,1
4,366.0,392.0,228379.0,228429.0,612.0,46.0,52.0,71337.0,103.0,127.0,1687.0,200.0,TypeOfSteel_A400,1
5,837.0,850.0,231429.0,231443.0,155.0,13.0,14.0,16093.0,55.0,134.0,1687.0,200.0,TypeOfSteel_A400,1
6,390.0,402.0,2513153.0,2513182.0,247.0,14.0,29.0,26419.0,NaN,126.0,1387.0,50.0,TypeOfSteel_A400,1
7,1351.0,1360.0,4807459.0,4807479.0,135.0,12.0,21.0,13096.0,NaN,109.0,1387.0,50.0,TypeOfSteel_A400,1
8,1325.0,1336.0,4848223.0,4848269.0,376.0,13.0,47.0,37703.0,NaN,117.0,1387.0,50.0,TypeOfSteel_A400,1
9,542.0,564.0,51943.0,51952.0,132.0,32.0,20.0,14760.0,104.0,119.0,1227.0,40.0,TypeOfSteel_A400,1


Como podemos ver acima, temos valores numéricos, categóricos string e o nosso target é binário. No dataframe acima já podemos ver que existem valores nulos, que devem ser tratados antes do treinamento.

In [ ]:
df.describe()

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,defect_type
count,967.000000,967.000000,9.670000e+02,9.670000e+02,967.000000,967.000000,967.000000,9.670000e+02,729.000000,967.000000,967.000000,967.000000,967.000000
mean,723.846949,748.890383,1.674985e+06,1.675013e+06,467.441572,44.491210,36.544984,4.957048e+04,95.779150,129.839710,1498.494312,92.645295,0.373320
std,494.305219,492.757755,1.892184e+06,1.892181e+06,1733.723575,85.896681,59.870839,1.814585e+05,28.647619,19.304366,158.154829,62.971298,0.483936
min,0.000000,4.000000,6.712000e+03,6.724000e+03,15.000000,4.000000,2.000000,1.826000e+03,0.000000,37.000000,1227.000000,40.000000,0.000000
25%,253.000000,283.000000,3.959620e+05,3.960710e+05,80.000000,14.000000,12.000000,9.005500e+03,77.000000,124.000000,1360.000000,40.000000,0.000000
50%,753.000000,778.000000,1.046565e+06,1.046583e+06,133.000000,21.000000,19.000000,1.489600e+04,101.000000,127.000000,1373.000000,70.000000,0.000000
75%,1122.500000,1141.500000,2.326936e+06,2.326952e+06,261.500000,41.000000,34.000000,2.872150e+04,112.000000,135.000000,1687.000000,100.000000,1.000000
max,1705.000000,1713.000000,1.280650e+07,1.280652e+07,37334.000000,1275.000000,903.000000,3.918209e+06,203.000000,253.000000,1794.000000,300.000000,1.000000


In [ ]:
df.nunique().sort_values()

defect_type               2
type_of_steel             4
slab_thickness           23
conveyer_width           75
max_pixel_luminosity     84
min_pixel_luminosity    115
slab_length             142
slab_width              151
area_pixels             420
min_x_defect            705
max_x_defect            706
sum_pixel_luminosity    951
min_y_defect            966
max_y_defect            966
dtype: int64

Podemos ver aqui que a coluna *'type_of_steel'* está com o número incorreto de dados únicos, já que temos apenas duas categorias de placa. Mais um dado que terá que ser analisado e tratado.

In [ ]:
df_cont = df["defect_type"].value_counts()

print('Em valores absolutos:')
display(df_cont)

print('\nEm percentual:')
display(df_cont / df.shape[0] * 100)
print(f'\n A taxa de desbalanceamento é de {df_cont[0]/df_cont[1]:.2f}.')

px.bar(df_cont)

Em valores absolutos:


0    606
1    361
Name: defect_type, dtype: int64


Em percentual:


0    62.668046
1    37.331954
Name: defect_type, dtype: float64


 A taxa de desbalanceamento é de 1.68.


## Tratamento de Dados


### Tratando dados incorretos

Na descrição das colunas está que são apenas duas categorias de placa, mas na descrição do nosso dataframe temos 4 tipos de dados. Sendo assim, vamos imprimir as categorias existentes nessa coluna e verificar as inconsistências.

In [ ]:
df["type_of_steel"].unique()

array(['TypeOfSteel_A300', 'TypeOfSteel_A400', 'TypeOfStel_A300',
       'TypeOfSteel_????'], dtype=object)

Como imaginado, temos um valor que está escrito incorretamente, mas é simples de resolver, e um valor TypeOfSteel_????, em que não se sabe o tipo da placa. Sendo assim, como não é viavel escolher uma placa qualquer para esse dado, pois não tenho conhecimento para decidir esse valor e julga-lo correto e nem posso realizar equações matemáticas com ele, a opção será remover a linha do dado.

In [ ]:
df.loc[df["type_of_steel"] == 'TypeOfSteel_????']

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type


In [ ]:
df.loc[df["type_of_steel"] == 'TypeOfSteel_????', "type_of_steel"] = None
df.loc[df["type_of_steel"].isna()]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [ ]:
df = df.dropna(subset=['type_of_steel'])
df.loc[df["type_of_steel"].isna()]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type


Por fim, corrigir os dados escritos de forma incorreta.

In [ ]:
df.loc[df["type_of_steel"] == 'TypeOfStel_A300', ["type_of_steel"]] ='TypeOfSteel_A300'

In [ ]:
df["type_of_steel"].unique()

array(['TypeOfSteel_A300', 'TypeOfSteel_A400'], dtype=object)

### Tratando Nulos
Abaixo pode-se notar que a coluna *'min_pixel_luminosity'* tem valores nulos.



In [ ]:
print(df.isna().sum(axis=0))

min_x_defect              0
max_x_defect              0
min_y_defect              0
max_y_defect              0
area_pixels               0
slab_width                0
slab_length               0
sum_pixel_luminosity      0
min_pixel_luminosity    233
max_pixel_luminosity      0
conveyer_width            0
slab_thickness            0
type_of_steel             0
defect_type               0
dtype: int64


In [ ]:
df.loc[df.isna().sum(axis=1) > 0, :]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
1,1252.0,1348.0,355940.0,356016.0,1812.0,119.0,135.0,196003.0,NaN,132.0,1687.0,80.0,TypeOfSteel_A300,1.0
6,390.0,402.0,2513153.0,2513182.0,247.0,14.0,29.0,26419.0,NaN,126.0,1387.0,50.0,TypeOfSteel_A400,1.0
7,1351.0,1360.0,4807459.0,4807479.0,135.0,12.0,21.0,13096.0,NaN,109.0,1387.0,50.0,TypeOfSteel_A400,1.0
8,1325.0,1336.0,4848223.0,4848269.0,376.0,13.0,47.0,37703.0,NaN,117.0,1387.0,50.0,TypeOfSteel_A400,1.0
11,932.0,943.0,3020344.0,3020360.0,128.0,12.0,16.0,13996.0,NaN,126.0,1687.0,50.0,TypeOfSteel_A400,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,1662.0,1670.0,2729960.0,2729972.0,55.0,11.0,12.0,4727.0,NaN,93.0,1690.0,70.0,TypeOfSteel_A300,0.0
903,1253.0,1263.0,2860158.0,2860168.0,76.0,13.0,10.0,7943.0,NaN,125.0,1688.0,70.0,TypeOfSteel_A300,0.0
905,109.0,141.0,4025621.0,4025624.0,53.0,32.0,7.0,6578.0,NaN,159.0,1690.0,70.0,TypeOfSteel_A300,0.0
907,1087.0,1119.0,80993.0,80996.0,64.0,32.0,7.0,9247.0,NaN,190.0,1692.0,70.0,TypeOfSteel_A300,0.0


Para resolver o problema devemos excluir as linhas? - 'Depende'... haha

Visto que são muitos valores e é apenas uma coluna das linhas que está sem o valor, podemos substituir os valores. Mas pelo que?
Existem algumas soluções possíveis:
- Pela média dos valores da coluna
- Pelos mediana
- Pelos valores mais frequêntes

A opção escolhida por mim foi a substituição pelos valores mais frequentes. Para não repetir os valores 233 vezes e também escolher melhor o valor de substituição, eu correlacionei a coluna *'min_pixel_luminosity'* com a coluna *'type_of_steel'*, obtendo o valor mais frequente de cada categoria e substituindo nas linhas nulas do respectivo tipo.


In [ ]:
null_values = df.loc[df.isna().sum(axis=1) > 0, :] # Anotando os indices nulos para verificarmos eles depois

In [ ]:
px.line(df.groupby(['min_pixel_luminosity', 'type_of_steel']).size().reset_index(name='count'),
              x='min_pixel_luminosity', y='count', color='type_of_steel') # Analisando de forma de visual os valores mais frequentes

In [ ]:
from sklearn.impute import SimpleImputer

classes = df['type_of_steel'].unique()

for classe in classes:
    subset = df[df['type_of_steel'] == classe]
    simp = SimpleImputer(strategy='most_frequent')
    subset = simp.fit_transform(subset)
    df.loc[df['type_of_steel'] == classe] = subset



In [ ]:
df.loc[df.isna().sum(axis=1) > 0, :]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type


In [ ]:
# Verificando como ficaram os valores antigos
df.loc[null_values.index, :][['min_pixel_luminosity', 'type_of_steel']]

,min_pixel_luminosity,type_of_steel
1,77.0,TypeOfSteel_A300
6,104.0,TypeOfSteel_A400
7,104.0,TypeOfSteel_A400
8,104.0,TypeOfSteel_A400
11,104.0,TypeOfSteel_A400
...,...,...
901,77.0,TypeOfSteel_A300
903,77.0,TypeOfSteel_A300
905,77.0,TypeOfSteel_A300
907,77.0,TypeOfSteel_A300


## Tratamento de Outliers

Incio plotando o bloxpot dos valores das colunas para analisa-los.

In [ ]:
colunas = df.columns.tolist()
colunas.remove('type_of_steel')
colunas.remove('defect_type')

for var in colunas:
    fig = px.box(data_frame=df, x=var, points='all', orientation='h')
    fig.update_layout(height=300)
    fig.show()

**Conclusão alcançada sobre os valores após avaliar os gráficos:**

Esses dois valores não apresentaram nenhum outlier.
- min_x_defect
- max_x_ defect

A partir daqui é notavel varios outliers. É necessário exclui-los? - Depende 😅

É necessário avaliar cada caso, e também verificar com os profissionais da área que irão usar o sistema, se esse dado é valido ou se realmente não é aplicavel.

Essas duas colunas possuem valores muito maiores que os dois anteriores. Como a maior parte dos valores está na casa do milhões, ao analisar acredito que os outliers possam ser valores possíveis, e possível também que esses valores apenas estejam em uma unidade de medida diferente, como micrometro ou nanometro. Essas duvidas podem ser sanadas em uma conversa com especialistas na área.

- min_y_ defect
- max_y_ defect


Esse valor varia de acordo com o tamanho do placa, e levando em consideração que existem valores de x e y próximos da casa dos mil, é valido que exista area em pixels com valores altos de área de pixels.
- area_pixels – Total de pixels presentes na placa

Essa informação se interliga com as outras acima, um tamanho maior de placa afirma um valor maior das outras colunas. Porém, não é possível afirmar que esses outliers são inválidos sem a ajuda de um especialista do setor.
- slab_width – Largura da placa (eixo X)
- slab_length – Comprimento da placa (eixo Y)
- slab_thickness - Grossura ou Densidade da placa (?)

O gráfico mostra outliers, mas quando tratamos de pixels e luminosidade, tratamos de uma variação de valores entre 0 e 255. Sendo 0 preto, e 255 branco.
Sendo assim, os outliers exibidos no boxsplot dessa coluna são válidos.
- min_pixel_luminosity
- max_pixel_luminosity

Podendo ter placas maiores e áreas maiores de pixels variando até 255, podemos afirmar que os outliers dessa coluna são possíveis e válidos.
Exemplo: 37.334 (maior área de pixels) x 252 (maior valor de luminosidade) = 9.408.168
Esse seria um valor máximo hipotético. E o outlier apresentado na coluna abaixo é de aprox. 3M. Podendo ser um valor possível.

- sum_pixel_luminosity – Soma da luminosidade dos pixels

Essa coluna não possui outliers.
- conveyer_width – Largura da esteira (correia) transportadora (eixo X)

In [ ]:
df.loc[df['max_y_defect']>12000000]

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
345,889.0,921.0,12438460.0,12438491.0,699.0,38.0,32.0,79690.0,104.0,132.0,1360.0,40.0,TypeOfSteel_A400,1
346,1094.0,1124.0,12806495.0,12806520.0,571.0,37.0,25.0,58587.0,63.0,127.0,1362.0,40.0,TypeOfSteel_A400,1
548,1135.0,1162.0,12416454.0,12416473.0,305.0,33.0,27.0,34579.0,104.0,127.0,1360.0,40.0,TypeOfSteel_A400,0


No dataframe acima estão os maiores valores dentre os outliers do *'max_y_defect'*. Nele podemos ver a relação entre os valores como falado anteriormente.

## Encoding
A seguir, é necessário antes de iniciar o treinamento, realizar o enconding para a classe *'type_of_steel'*

In [ ]:
df.nunique().sort_values()

defect_type               2
type_of_steel             4
slab_thickness           23
conveyer_width           75
max_pixel_luminosity     84
min_pixel_luminosity    115
slab_length             142
slab_width              151
area_pixels             420
min_x_defect            705
max_x_defect            706
sum_pixel_luminosity    951
min_y_defect            966
max_y_defect            966
dtype: int64

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder(dtype=int)
ord_enc.fit(df[['type_of_steel']])

df_enc = df.copy()
df_enc[['type_of_steel']] = ord_enc.transform(df[['type_of_steel']])
df_enc

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,type_of_steel,defect_type
0,38.0,49.0,735612.0,735624.0,113.0,11.0,12.0,12652.0,93.0,130.0,1707.0,100.0,0,1.0
1,1252.0,1348.0,355940.0,356016.0,1812.0,119.0,135.0,196003.0,77.0,132.0,1687.0,80.0,0,1.0
2,193.0,210.0,612201.0,612252.0,588.0,18.0,51.0,62182.0,73.0,135.0,1353.0,290.0,1,1.0
3,1159.0,1170.0,32914.0,32926.0,106.0,11.0,12.0,12792.0,100.0,134.0,1353.0,185.0,1,1.0
4,366.0,392.0,228379.0,228429.0,612.0,46.0,52.0,71337.0,103.0,127.0,1687.0,200.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,137.0,170.0,301492.0,301511.0,304.0,59.0,26.0,35778.0,111.0,126.0,1360.0,40.0,1,0.0
963,238.0,287.0,315114.0,315142.0,671.0,91.0,39.0,86424.0,119.0,143.0,1360.0,40.0,1,0.0
964,144.0,175.0,340581.0,340598.0,287.0,44.0,24.0,34599.0,112.0,133.0,1360.0,40.0,1,0.0
965,145.0,174.0,386779.0,386794.0,292.0,40.0,22.0,37572.0,120.0,140.0,1360.0,40.0,1,0.0


In [ ]:
df_enc = pd.get_dummies(df_enc, columns=['type_of_steel'], drop_first=True)
df_enc

,min_x_defect,max_x_defect,min_y_defect,max_y_defect,area_pixels,slab_width,slab_length,sum_pixel_luminosity,min_pixel_luminosity,max_pixel_luminosity,conveyer_width,slab_thickness,defect_type,type_of_steel_1
0,38.0,49.0,735612.0,735624.0,113.0,11.0,12.0,12652.0,93.0,130.0,1707.0,100.0,1.0,0
1,1252.0,1348.0,355940.0,356016.0,1812.0,119.0,135.0,196003.0,77.0,132.0,1687.0,80.0,1.0,0
2,193.0,210.0,612201.0,612252.0,588.0,18.0,51.0,62182.0,73.0,135.0,1353.0,290.0,1.0,1
3,1159.0,1170.0,32914.0,32926.0,106.0,11.0,12.0,12792.0,100.0,134.0,1353.0,185.0,1.0,1
4,366.0,392.0,228379.0,228429.0,612.0,46.0,52.0,71337.0,103.0,127.0,1687.0,200.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,137.0,170.0,301492.0,301511.0,304.0,59.0,26.0,35778.0,111.0,126.0,1360.0,40.0,0.0,1
963,238.0,287.0,315114.0,315142.0,671.0,91.0,39.0,86424.0,119.0,143.0,1360.0,40.0,0.0,1
964,144.0,175.0,340581.0,340598.0,287.0,44.0,24.0,34599.0,112.0,133.0,1360.0,40.0,0.0,1
965,145.0,174.0,386779.0,386794.0,292.0,40.0,22.0,37572.0,120.0,140.0,1360.0,40.0,0.0,1


## Separação treino-teste
Agora com os dados pré-processados, estão prontos para serem dividos em treino e teste, e também em folds.

In [ ]:
from sklearn.model_selection import StratifiedKFold # KFold se não houver diferença entre grupos

def split_data(df_enc, N_folds = 5):
  # Escolha do número de folds
  N_folds = 5

  # Criação do Splitter
  splitter = StratifiedKFold(
      n_splits=N_folds,
      random_state=38,
      shuffle=True)

  # Separar X do y
  df_sem_y = df_enc.copy().drop(columns='defect_type')
  df_y = df_enc.copy()['defect_type'].to_frame()

  return splitter, df_sem_y, df_y

In [ ]:
N_folds = 5
splitter, df_sem_y, df_y = split_data(df_enc, N_folds)

# Listas para armazenarem nossos grupos da validação cruzada

X_train_fold = []
y_train_fold = []
X_test_fold = []
y_test_fold = []

for index_train, index_test in splitter.split(df_sem_y, df_y):
  X_train = df_sem_y.iloc[index_train]
  y_train = df_y.iloc[index_train]
  X_test = df_sem_y.iloc[index_test]
  y_test =  df_y.iloc[index_test]

  # adicionar na lista
  X_train_fold.append(X_train)
  y_train_fold.append(y_train)
  X_test_fold.append(X_test)
  y_test_fold.append(y_test)

## Rescaling dos dados e treino do modelo

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression

# Esse é o nosso dicionário que vai conter as métricas do treino
train_metrics = {
    'accuracy': [],
    'recall': [],
    'precision': [],
    'f1': []
}

# Esse é o nosso dicionário que vai conter as métricas do teste
test_metrics = {
    'accuracy': [],
    'recall': [],
    'precision': [],
    'f1': []
}

# Listas para guardar as previsões feitas pro grupo de treino e teste
y_hat_train_fold = []
y_hat_test_fold = []
y_hat_test_proba_fold = []

# Executar para cada fold
for fold in range(N_folds):

  # Recuperando os dados desse fold específico
  X_train = X_train_fold[fold]
  X_test = X_test_fold[fold]
  y_train = y_train_fold[fold]
  y_test = y_test_fold[fold]

  # Criando o scaler aqui dentro para só escalar nos dados de treino
  x_scaler = StandardScaler()
  x_scaler.fit(X_train)

  # Aplicar a normalização
  X_train_norm = x_scaler.transform(X_train)
  X_test_norm = x_scaler.transform(X_test)

  # Treinar o modelo
  model = LogisticRegression(
      fit_intercept=False,
      random_state=38
  )
  model.fit(X_train_norm, y_train.iloc[:,0])

  # Realizar as previsões
  y_hat_train = model.predict(X_train_norm)
  y_hat_test = model.predict(X_test_norm)
  y_hat_proba_test = model.predict_proba(X_test_norm)[:,1]

  # Salvar previsões para o conjunto de treino
  y_hat_train_fold.append(y_hat_train)
  y_hat_test_fold.append(y_hat_test)
  y_hat_test_proba_fold.append(y_hat_proba_test)

  # Calcular métricas do treino
  acc = accuracy_score(y_train, y_hat_train)
  train_metrics['accuracy'].append(acc)

  rec = recall_score(y_train, y_hat_train)
  train_metrics['recall'].append(rec)

  precision = precision_score(y_train, y_hat_train)
  train_metrics['precision'].append(precision)

  f1 = f1_score(y_train, y_hat_train)
  train_metrics['f1'].append(f1)

  # Calcular métricas do teste
  acc = accuracy_score(y_test, y_hat_test)
  test_metrics['accuracy'].append(acc)

  rec = recall_score(y_test, y_hat_test)
  test_metrics['recall'].append(rec)

  precision = precision_score(y_test, y_hat_test)
  test_metrics['precision'].append(precision)

  f1 = f1_score(y_test, y_hat_test)
  test_metrics['f1'].append(f1)




## Avaliação de métricas

In [ ]:
df_train_metrics = pd.DataFrame(train_metrics)
df_train_metrics

,accuracy,recall,precision,f1
0,0.673629,0.761404,0.543860,0.634503
1,0.677546,0.800000,0.545455,0.648649
2,0.665796,0.744755,0.537879,0.624633
3,0.662321,0.762238,0.533007,0.627338
4,0.671447,0.762238,0.542289,0.633721


In [ ]:
df_train_metrics = df_train_metrics.mean(axis=0)
df_train_metrics

accuracy     0.670148
recall       0.766127
precision    0.540498
f1           0.633769
dtype: float64

In [ ]:
df_test_metrics = pd.DataFrame(test_metrics)
df_test_metrics

,accuracy,recall,precision,f1
0,0.625000,0.736111,0.500000,0.595506
1,0.671875,0.694444,0.549451,0.613497
2,0.656250,0.788732,0.523364,0.629213
3,0.659686,0.788732,0.528302,0.632768
4,0.664921,0.760563,0.534653,0.627907


In [ ]:
df_test_metrics = df_test_metrics.mean(axis=0)
df_test_metrics

accuracy     0.655546
recall       0.753717
precision    0.527154
f1           0.619778
dtype: float64

In [ ]:
df_metrics = pd.concat([df_train_metrics, df_test_metrics], axis=1)
df_metrics = df_metrics.rename(columns={0: 'Treino', 1: 'Teste'}).round(3)
df_metrics

,Treino,Teste
accuracy,0.670,0.656
recall,0.766,0.754
precision,0.540,0.527
f1,0.634,0.620


## Matriz de Confusão

In [ ]:
y_test_list = []
for y_fold in y_test_fold:
  y_test_list.extend(y_fold.iloc[:,0])

y_hat_test_list = []
for lista_fold in y_hat_test_fold:
  y_hat_test_list.extend(lista_fold)

y_test_hat_proba_list = []
for lista_fold in y_hat_test_proba_fold:
  y_test_hat_proba_list.extend(lista_fold)

In [ ]:
confusion_matrix = pd.DataFrame(
    sklearn.metrics.confusion_matrix(y_test_list, y_hat_test_list),
    index=['Defeito 0', 'Defeito 1'],
    columns=['Defeito 0', ' Defeito 1'],
)

display(confusion_matrix.style.background_gradient(axis=None))

,Defeito 0,Defeito 1
Defeito 0,359,242
Defeito 1,88,269


Podemos analisar que o modelo obteve resultados visivelmente razoaveis, mas que podem-ser melhorados. E também pode ser um modelo excelente para o uso, dependendo de onde será aplicado, trazendo bons retornos (Isso será analisado depois de acordo com as regras de negócio fornecidas no exercicio 1).

# Ajustando threshold
Utilizando como métrica de avaliação de desempenho principal o F1, podemos ajustar o threshold a fim de aumentar o valor de resultado obtido.

In [ ]:
import numpy as np
# Exemplo de ajuste do limiar de decisão
thresholds = np.arange(0.01, 0.95, 0.01)

f1_scores = []
for threshold in thresholds:
  y_pred_th = (y_test_hat_proba_list > threshold).astype(int)
  f1 = f1_score(y_test_list, y_pred_th)
  f1_scores.append(f1)

optimal_threshold = thresholds[np.argmax(f1_scores)]
print("Limiar de Decisão Ótimo:", optimal_threshold)

Limiar de Decisão Ótimo: 0.36000000000000004


Ao avaliar valores entre 0.01 e 0.95, obtivemos que o valor 0.36 obteve o melhor f1.

In [ ]:
df_th = pd.DataFrame(
    data={
        'Threshold': thresholds,
        'F1': f1_scores
    }
)
df_th

,Threshold,F1
0,0.01,0.543675
1,0.02,0.542986
2,0.03,0.545042
3,0.04,0.546697
4,0.05,0.549199
...,...,...
89,0.90,0.053908
90,0.91,0.053908
91,0.92,0.053908
92,0.93,0.053908


In [ ]:
px.line(df_th,x='Threshold',y='F1',title='F1 por Threshold',markers=True)

Gerando novas predições com base nesse threshold e nos valores probabilisticos gerados pelo modelo, temos como resultado:

In [ ]:
from sklearn.metrics import confusion_matrix

# Para testes manuais
y_test_th = (np.array(y_test_hat_proba_list) > 0.36).astype(int)

# Criação da matriz de confusão
confusion_matrix = pd.DataFrame(
    confusion_matrix(y_test_list, y_test_th),
    index=['Defeito 0', 'Defeito 1'],
    columns=['Defeito 0', ' Defeito 1'],
)

# Mostra F1
f1 = f1_score(
    y_true=y_test_list,
    y_pred=y_test_th,
    average='macro'
    )
print('F1:', f'{f1:.3f}')

# Mostra a matriz de confusão
display(confusion_matrix.style.background_gradient(axis=None))

F1: 0.518


,Defeito 0,Defeito 1
Defeito 0,180,421
Defeito 1,30,327


## Ajustando o threshold com base no problema a se resolver

In [ ]:
from sklearn.metrics import confusion_matrix

# Para testes manuais
y_test_th = (np.array(y_test_hat_proba_list) > 0.65).astype(int)

# Grid-Search

O modelo também pode ser melhorado testando novas combinações de hyperparâmetros. Para isso, utilizei o Grid-Search com alguns parâmetros do modelo de Regressão Logistica.

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'penalty': ['l2'],
              'class_weight': [None, 'balanced']
              }

Sendo...

- 'C': Parâmetro que controla a força da regularização, sendo valores menores mais fortemente regularizados e valores maiores menos regularizados.

- 'penalty': Especifica o tipo de regularização, com 'l2' indicando a penalização dos coeficientes quadrados para evitar overfitting.

- 'class_weight': Atribui pesos às classes, sendo 'balanced' útil para lidar automaticamente com desbalanceamento proporcional às frequências das classes

In [ ]:
from sklearn.model_selection import train_test_split

# Considere df como o seu DataFrame
X = df_enc.drop('defect_type', axis=1)
y = df_enc['defect_type']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression

# Criando o scaler aqui dentro para só escalar nos dados de treino
x_scaler = StandardScaler()
x_scaler.fit(X_train)

# Aplicar a normalização
X_train_norm = x_scaler.transform(X_train)


# Treinar o modelo
model = LogisticRegression(
    random_state=38,
    max_iter=1000
)

param_grid = {'C': [ 0.001, 0.01, 0.1, 1, 10, 100, 500, 800, 1000],
              'penalty': ['l2'],
              'class_weight': [None, 'balanced']
              }
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_norm, y_train)
best_params = grid_search.best_params_
best_params

{'C': 500, 'class_weight': 'balanced', 'penalty': 'l2'}

In [ ]:
N_folds = 5
splitter, df_sem_y, df_y = split_data(df_enc, N_folds)

# Listas para armazenarem nossos grupos da validação cruzada

X_train_fold = []
y_train_fold = []
X_test_fold = []
y_test_fold = []

for index_train, index_test in splitter.split(df_sem_y, df_y):
  X_train = df_sem_y.iloc[index_train]
  y_train = df_y.iloc[index_train]
  X_test = df_sem_y.iloc[index_test]
  y_test =  df_y.iloc[index_test]

  # adicionar na lista
  X_train_fold.append(X_train)
  y_train_fold.append(y_train)
  X_test_fold.append(X_test)
  y_test_fold.append(y_test)

Treinando o modelo com os novos parâmetros.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression

# Esse é o nosso dicionário que vai conter as métricas do treino
train_metrics = {
    'accuracy': [],
    'recall': [],
    'precision': [],
    'f1': []
}

# Esse é o nosso dicionário que vai conter as métricas do teste
test_metrics = {
    'accuracy': [],
    'recall': [],
    'precision': [],
    'f1': []
}

# Listas para guardar as previsões feitas pro grupo de treino e teste
y_hat_train_fold = []
y_hat_test_fold = []
y_hat_test_proba_fold = []

# Executar para cada fold
for fold in range(N_folds):

  # Recuperando os dados desse fold específico
  X_train = X_train_fold[fold]
  X_test = X_test_fold[fold]
  y_train = y_train_fold[fold]
  y_test = y_test_fold[fold]

  # Criando o scaler aqui dentro para só escalar nos dados de treino
  x_scaler = StandardScaler()
  x_scaler.fit(X_train)

  # Aplicar a normalização
  X_train_norm = x_scaler.transform(X_train)
  X_test_norm = x_scaler.transform(X_test)

  # Treinar o modelo
  model = LogisticRegression(
      fit_intercept=False,
      random_state=38,
      C=500,
      class_weight='balanced',
      penalty='l2'

  )

  model.fit(X_train_norm, y_train.iloc[:,0])

  # Realizar as previsões
  y_hat_train = model.predict(X_train_norm)
  y_hat_test = model.predict(X_test_norm)
  y_hat_proba_test = model.predict_proba(X_test_norm)[:,1]

  # Salvar previsões para o conjunto de treino
  y_hat_train_fold.append(y_hat_train)
  y_hat_test_fold.append(y_hat_test)
  y_hat_test_proba_fold.append(y_hat_proba_test)

  # Calcular métricas do treino
  acc = accuracy_score(y_train, y_hat_train)
  train_metrics['accuracy'].append(acc)

  rec = recall_score(y_train, y_hat_train)
  train_metrics['recall'].append(rec)

  precision = precision_score(y_train, y_hat_train)
  train_metrics['precision'].append(precision)

  f1 = f1_score(y_train, y_hat_train)
  train_metrics['f1'].append(f1)

  # Calcular métricas do teste
  acc = accuracy_score(y_test, y_hat_test)
  test_metrics['accuracy'].append(acc)

  rec = recall_score(y_test, y_hat_test)
  test_metrics['recall'].append(rec)

  precision = precision_score(y_test, y_hat_test)
  test_metrics['precision'].append(precision)

  f1 = f1_score(y_test, y_hat_test)
  test_metrics['f1'].append(f1)




In [ ]:
df_train_metrics = pd.DataFrame(train_metrics)
df_train_metrics = df_train_metrics.mean(axis=0)
df_test_metrics = pd.DataFrame(test_metrics)
df_test_metrics = df_test_metrics.mean(axis=0)
df_metrics = pd.concat([df_train_metrics, df_test_metrics], axis=1)
df_metrics = df_metrics.rename(columns={0: 'Treino', 1: 'Teste'}).round(3)
df_metrics

,Treino,Teste
accuracy,0.684,0.668
recall,0.810,0.785
precision,0.552,0.538
f1,0.657,0.638
